In [ ]:
# Necessary Libraries
import pandas as pd
import numpy as np
import networkx as nx
import json
from pprint import pprint

In [ ]:
# Helper maps/dictionaries

# Map from string to its numerical weight
edge_weight_map = { '++': 2, '+': 1, '0': 0, '-': -1, '--': -2 }

# Data type of edge in graph
dt_edge = [('weight', int),
           ('weight_absolute', int), 
           ('strengthen', int), 
           ('weaken', int), 
           ('sign', int)]

# Maps string to dt_edge quintuple
def edge_map(w):
    w = edge_weight_map[w]
    abs_w = abs(w)
    sign = np.sign(w)
    return (w, abs_w, (abs_w if sign == 1 else 0), (abs_w if sign == -1 else 0), sign)

In [ ]:
# Extracts action systems from loaded data
def get_action_systems(data):
    entries = {}
    
    for i, entry in enumerate(data):
        entries[entry['id']] = entry['name']
    
    return entries

def get_life_entries(data, action_systems):
    identifier = {}
    labels = {}
    node_action_systems_id = {}
    node_action_systems = {}
    influences = {}
    
    for i, entry in enumerate(data):
        identifier[i] = entry['id']
        labels[i] = entry['title']
        node_action_systems_id[i] = entry['actionSystemId']
        node_action_systems[i] = action_systems[entry['actionSystemId']]
        influences[i] = entry['influence']
    
    return identifier, labels, node_action_systems_id, node_action_systems, influences

# Loads the json and creates a networkx multidi graph
def json_to_graph(path, encoding='utf-8', verbose=False):
    json_data = None
    action_systems = {}
    labels = {}
    node_action_systems = {}
    influences = {}
    
    with open(path, encoding=encoding) as data_file:
        json_data = json.load(data_file)
    
    # Print the loaded json data
    if verbose == True:
        ppriint(json_data)
    
    # Gets adjacency matrix and maps it to the dt_edge data type
    df = pd.DataFrame(json_data['connections'])
    df = df.applymap(edge_map)
    am = np.array(df, dtype=dt_edge)
    
    if verbose == True:
        # Print mapped adjacency matrix
        print(am)
    
    # Get action systems
    action_systems = get_action_systems(json_data['actionSystems'])
    
    # Get life entries
    identifier, labels, node_action_systems_id, node_action_systems, influences = get_life_entries(json_data['lifeEntries'], action_systems)
    
    # Create graph by loading the adjacency matrix
    G = nx.from_numpy_matrix(am, parallel_edges=False, create_using=nx.MultiDiGraph())
    
    # Set attributes on nodes
    nx.set_node_attributes(G, identifier, name='id')
    nx.set_node_attributes(G, labels, name='label')
    nx.set_node_attributes(G, node_action_systems_id, name='actionSystemId')
    nx.set_node_attributes(G, node_action_systems, name='actionSystem')
    nx.set_node_attributes(G, influences, name='influence')
    
    return G

def to_d3_json(G):
    nodes = []
    links = []
    
    labels = nx.get_node_attributes(G, name='label')
    influences = nx.get_node_attributes(G, name='influence')
    action_systems = nx.get_node_attributes(G, name='actionSystem')
    
    weights = nx.get_edge_attributes(G, name='weight')
    weight_absolute = nx.get_edge_attributes(G, name='weight_absolute')
    strengthen = nx.get_edge_attributes(G, name='strengthen')
    weaken = nx.get_edge_attributes(G, name='weaken')
    sign = nx.get_edge_attributes(G, name='sign')
    
    for i in range(len(labels)):
        nodes.append({
            'label': labels[i],
            'influence': influences[i],
            'actionSystem': action_systems[i]
        })
    
    print(weights)
    print(strengthen)

In [ ]:
G = json_to_graph('data/dataset_20180403/180331_Herbert_Heim.json')
to_d3_json(G)